# 🧩 Célula 1 — Importações e carregamento dos dados

In [ ]:
# 🧠 Data Preprocessing — Módulo 01
# Dataset: cars.csv
# Autor: Ari Soares

# 1️⃣ Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Carregar dataset
df = pd.read_csv('**Caminho**/cars.csv')

# Verificar estrutura inicial
print('Verificação de estrutura inicial:\n')
df.info()
df.head()

Verificação de estrutura inicial:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mpg          261 non-null    float64
 1   cylinders    261 non-null    int64  
 2   cubicinches  261 non-null    object 
 3   hp           261 non-null    int64  
 4   weightlbs    261 non-null    object 
 5   time-to-60   261 non-null    int64  
 6   year         261 non-null    int64  
 7   brand        261 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 16.4+ KB


,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year,brand
0,14.0,8,350,165,4209,12,1972,US.
1,31.9,4,89,71,1925,14,1980,Europe.
2,17.0,8,302,140,3449,11,1971,US.
3,15.0,8,400,150,3761,10,1971,US.
4,30.5,4,98,63,2051,17,1978,US.


# 🔧 Célula 2 — Correção de tipos incorretos

In [24]:
# 2️⃣ Corrigir colunas com tipo incorreto (object → float)

cols_to_fix = ['cubicinches', 'weightlbs']

for col in cols_to_fix:
    df[col] = df[col].replace({',': '', ' ': np.nan, '': np.nan}, regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Substituir valores ausentes pela média
for col in cols_to_fix:
    df[col].fillna(df[col].mean(), inplace=True)

# Confirmar conversão
df[cols_to_fix].dtypes

/tmp/ipython-input-2968172770.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


,0
cubicinches,float64
weightlbs,float64


# 🔍 Célula 3 — Análise rápida e verificação

In [26]:
# 3️⃣ Inspeção dos dados após correção
print("Formato:", df.shape)
print("\nValores ausentes por coluna:\n", df.isnull().sum())

# Estatísticas descritivas
print("\nEstatísticas descritivas:\n")
df.describe().T

Formato: (261, 8)

Valores ausentes por coluna:
 mpg            0
cylinders      0
cubicinches    0
hp             0
weightlbs      0
time-to-60     0
year           0
brand          0
dtype: int64

Estatísticas descritivas:



,count,mean,std,min,25%,50%,75%,max
mpg,261.0,23.144828,7.823570,10.0,16.9,22.0,28.8,46.6
cylinders,261.0,5.590038,1.733310,3.0,4.0,6.0,8.0,8.0
cubicinches,261.0,200.918919,108.837269,68.0,101.0,156.0,302.0,455.0
hp,261.0,106.360153,40.499959,46.0,75.0,95.0,138.0,230.0
weightlbs,261.0,3009.833333,849.097575,1613.0,2254.0,2904.0,3664.0,4997.0
time-to-60,261.0,15.547893,2.910625,8.0,14.0,16.0,17.0,25.0
year,261.0,1976.819923,3.637696,1971.0,1974.0,1977.0,1980.0,1983.0


# ⚙️ Célula 4 — Definição das variáveis para o pré-processamento

In [27]:
# 4️⃣ Separar tipos de variáveis

# Variável alvo
target = 'mpg'

# Variáveis numéricas contínuas (para escala)
num_features = ['cubicinches', 'hp', 'weightlbs', 'time-to-60']

# Variáveis categóricas
cat_features = ['brand']

# Variáveis numéricas que NÃO serão escaladas
excluded_features = ['cylinders', 'year']

print("Numéricas (escaladas):", num_features)
print("Categóricas:", cat_features)
print("Excluídas da escala:", excluded_features)


Numéricas (escaladas): ['cubicinches', 'hp', 'weightlbs', 'time-to-60']
Categóricas: ['brand']
Excluídas da escala: ['cylinders', 'year']


# 🧠 Célula 5 — Criação do pipeline de pré-processamento

In [28]:
# 5️⃣ Criar transformadores separados para numéricas e categóricas

# Escalador para variáveis numéricas contínuas
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Codificador para variáveis categóricas
cat_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))
])

# Combinar tudo em um ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ],
    remainder='passthrough'  # mantém colunas não transformadas
)

preprocessor


ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['cubicinches', 'hp', 'weightlbs',
                                  'time-to-60']),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['brand'])])

# 🧪 Célula 6 — Aplicar o pipeline e gerar DataFrame transformado

In [29]:
# 6️⃣ Aplicar pré-processamento aos dados

# Aplicar transformações
X_transformed = preprocessor.fit_transform(df)

# Recuperar nomes das novas colunas codificadas
encoded_cols = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(cat_features)

# Construir novo DataFrame com todas as variáveis
processed_columns = num_features + list(encoded_cols) + excluded_features + [target]
df_processed = pd.DataFrame(X_transformed, columns=processed_columns)

# Exibir resultado
df_processed.head()


,cubicinches,hp,weightlbs,time-to-60,brand_ Japan.,brand_ US.,cylinders,year,mpg
0,1.372393,1.450681,1.414997,-1.221287,0.0,1.0,14.0,8.0,1972.0
1,-1.030290,-0.874768,-1.280086,-0.532829,0.0,0.0,31.9,4.0,1980.0
2,0.930520,0.832210,0.518209,-1.565516,0.0,1.0,17.0,8.0,1971.0
3,1.832677,1.079598,0.886364,-1.909745,0.0,1.0,15.0,8.0,1971.0
4,-0.947439,-1.072679,-1.131408,0.499857,0.0,1.0,30.5,4.0,1978.0


# 💾 Célula 7 — Salvar dados tratados

In [30]:
# 7️⃣ Salvar dataset pré-processado
df_processed.to_csv('cars_preprocessed.csv', index=False)
print("✅ Dataset salvo como cars_preprocessed.csv")

✅ Dataset salvo como cars_preprocessed.csv
